In [ ]:
# Run some setup code for this notebook.

import matplotlib.pyplot as plt


# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Load Dataset

In [ ]:
import pandas as pd
df = pd.read_csv('loan.csv')
df.head()

## Dataset Info

In [ ]:
df.info()

## Data Cleaning

In [ ]:
df.drop("Loan_ID", axis=1, inplace=True)
# Checking the Missing Values
df.isnull().sum()

In [ ]:
################################################################################
# TODO:                                                                        #
# Task1 deal with NULL rows, you can either choose to drop them or replace them with mean or other value #
################################################################################

df["Credit_History"] = df["Credit_History"].astype(str)

# fill numeric values with mean
mean = df.mean(numeric_only=True)
df = df.fillna(value=mean)

# fill string values with mode
mode = df.mode()
mode = {k: mode[k][0] for k in mode}
df = df.fillna(value=mode)

df.isnull().sum()

################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## Encode 

In [ ]:
################################################################################
# TODO:                                                                        #
# Task2 deal with categorical features
# Tip use pd.get_dummies. 
################################################################################

df = pd.get_dummies(df)

# drop useless columns for 2-value properties
df = df.drop(["Gender_Female", "Married_No", "Education_Not Graduate", "Self_Employed_No", "Loan_Status_N", "Credit_History_0.0"], axis=1)

df.head()

################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## Data process

In [ ]:
import numpy as np


def normalization(data):
    range = np.max(data) - np.min(data)
    return (data - np.min(data)) / range


# normalize data

for key in ["ApplicantIncome", "CoapplicantIncome", "LoanAmount", "Loan_Amount_Term"]:
    df[key] = np.sqrt(df[key])
    df[key] = normalization(df[key])

df.head()

In [ ]:
def split(df, r=0.8):
    df = df.sample(frac=1).reset_index(drop=True)
    train_size = int(df.shape[0] * r)
    train, test = df.loc[0:train_size], df.loc[train_size:]
    return train, test


def split_XY(df):
    df_X = df.drop(["Loan_Status_Y"], axis=1)
    df_Y = df["Loan_Status_Y"]
    return df_X, df_Y


################################################################################
# TODO:                                                                        #
# Task3 split the dataset into X_train, X_test, y_train, y_test
# Optional: you can also use normalization
################################################################################


df_Yes = df[df["Loan_Status_Y"] == 1]
df_No = df[df["Loan_Status_Y"] == 0]
train_Yes, test_Yes = split(df_Yes, 0.8)
train_No, test_No = split(df_No, 0.8)
train = pd.concat([train_Yes, train_No])
test = pd.concat([test_Yes, test_No])
train_X, train_Y = split_XY(test)
test_X, test_Y = split_XY(test)

train_X.head()

################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## Train

In [ ]:
from Logistic import LogisticRegression
import matplotlib.pyplot as plt

################################################################################
# TODO:                                                                        #
# Task4 train your model and plot the loss curve of training
# You need to complete the Logistic.py file
# model = LogisticRegression()
################################################################################

model = LogisticRegression(penalty="l2", gamma=2e-5, fit_intercept=True)

loss, acc = model.fit(
    train_X, train_Y, lr=0.01, tol=1e-8, max_iter=10**5, test_X=test_X, test_y=test_Y
)

plt.xlabel("epoch")
plt.ylabel("loss")
(h1,) = plt.plot(loss, "b", label="loss")
plt.twinx()
plt.ylabel("acc")
(h2,) = plt.plot(acc, "r", label="acc")
plt.legend(handles=[h1, h2], loc="best")

################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## Test

In [ ]:
################################################################################
# TODO:                                                                        #
# Task5 compare the accuracy(or other metrics you want) of test data with different parameters you train with
################################################################################

print(model.predict_acc(test_X, test_Y))


################################################################################
#                                 END OF YOUR CODE                             #
################################################################################